# Práctica 2: Aprendizaje y selección de modelos de clasificación* 

### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

\* Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo

### Grupo H:

* Alejandro Fernández Arjona
* Pablo Torrijos Arenas

En esta práctica estudiaremos los modelos más utilizados en `scikit-learn` para conocer los distintos hiperparámetros que los configuran y estudiar los clasificadores resultantes. Además, veremos métodos de selección de modelos orientados a obtener una configuración óptima de hiperparámetros.

---

En esta libreta vamos a realizar el aprendizaje y selección de modelos para la base de datos [Breast Cancer Wisconsin (Diagnostic) Data Set](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data).

El aprendizaje y selección de modelos para [Pima Indians Diabetes Database](https://www.kaggle.com/uciml/pima-indians-diabetes-database) se encuentra en el siguiente kernel de Kaggle: [Selección de modelos Pima](https://www.kaggle.com/alexfer/aprendizaje-de-modelos-diabetes-grupo-h)

Además, el estudio de la libreta [Hyperparameter Search Comparison (Grid vs Random)](https://www.kaggle.com/crawford/hyperparameter-search-comparison-grid-vs-random) se encuentra en: [Hyperparameter Search Comparison](https://www.kaggle.com/alexfer/hyperparameter-search-comparison-grid-vs-random)

# 1. Preliminares

Lo primero que tenemos que hacer es cargar las librerías para que estén disponibles posteriormente:

In [ ]:
# Third party
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

from imblearn import FunctionSampler
from imblearn.pipeline import make_pipeline

import numpy as np
import pandas as pd

# Librerías para las gráficas
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as pypl

# Local application
import miner_a_de_datos_aprendizaje_modelos_utilidad as utils
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils_ex

# Establecemos el máximo número de filas y columnas para la salida de Pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

Además, fijamos una semilla que hará que los experimentos sean reproducibles:

In [ ]:
random_state = 27912

# 2. Carga de datos

En este caso, vamos a utilizar el conjunto de datos `Breast Cancer Wisconsin (Diagnostic) Data Set`. creado por Dr. William H. Wolberg, W. Nick Street y Olvi L. Mangasarian y donada en 1995 a UCI Machine Learning Repository.

Según la descripción dada de la base de datos, los valores se calculan a partir de una imagen digitalizada de una aspiración mediantee aguja fina (FNA) de una masa mamaria. Describen las características de los núcleos celulares presentes en la imagen.

Las variables presentes en la base de datos son:

1. ID de la instancia (`id`).
2. Diagnóstico (`Diagnosis`). Es una variable binaria que toma los valores M = Maligno, B = Benigno. Esta será la variable clase.
3. 30 variables, correspondientes a la media, desviación típica y `worst` (media de los 3 peores valores) de 10 características de las imágenes reales. Estas variables están en el siguiente orden: primero las 10 medias, luego las 10 desviaciones típicas y por último los 10 valores `worst`.

Las 10 características son:

1. Radio (`radius`). Media de las distancias desde el centro a los puntos del perímetro.
2. Textura (`texture`). Desviación estándar de los valores de la escala de grises.
3. Perímetro (`perimeter`).
4. Area (`area`).
5. Suavidad (`smoothness`). Variación local de las longitudes de los radios.
6. Compactación (`compactness`). Perímetro$^2$ / área - 1.
7. Concavidad (`concavity`). Severidad de las porciones cóncavas del contorno.
8. Puntos cóncavos (`concave points`). Número de porciones cóncavas del contorno.
9. Simetría (`symmetry`).
10. Dimensión fractal (`fractal dimension`). "Aproximación a la costa" (`coastline approximation`) - 1.


In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"

data = utils.load_data(filepath, index, target)

Comprobando que se ha cargado correctamente:

In [ ]:
data.sample(5, random_state=random_state)

Podemos ver que se crea una variable cuyo nombre es `Unnamed 32`, al igual que ya vimos en la Práctica 1, ya que en el fichero `.csv` de la base de datos la línea con el nombre de las variables acaba con una coma, por lo que Pandas detecta una variable más sin ningún nombre especificado, y todos sus valores perdidos. Por ello, vamos a borrar esta última variable, y a comprobar que el borrado se realiza correctamente:

In [ ]:
data = data.drop(data.columns[[31]], axis='columns')

data.sample(5, random_state=random_state)

Ahora, dividimos nuestros datos en variables predictoras (`X`) y variable clase (`y`). Además, renombramos las intancias de la variable clase, sustituyendo `B` por 0 y `M` por 1. Esto lo hacemos para evitar problemas posteriores con métricas que solo utilizan valores numéricos como variable clase.

In [ ]:
(X, y) = utils.divide_dataset(data, target)
y = y.replace({"B": 0, "M": 1})

Y comprobamos que se han separado correctamente tanto las variables predictoras:

In [ ]:
X.sample(5, random_state=random_state)

Como la variable clase:

In [ ]:
y.sample(5, random_state=random_state)

Por último, dividimos el conjunto de datos en entrenamiento (70%) y prueba (30%) mediante un *holdout* estratificado. Este nuevo conjunto de entrenamiento nos servirá para visualizar los datos y realizar el análisis exploratorio y el preprocesamiento de la práctica 1, y posteriormente la selección de los modelos. Y el conjunto de prueba lo utilizaremos al final para obtener un resultado final que no haya sido sobreajustado en la selección de los modelos.

In [ ]:
stratify = y
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)

Y nos aseguramos que se ha realizado adecuadamente, tanto el conjunto de datos de entrenamiento:

In [ ]:
X_train.sample(5, random_state=random_state)

In [ ]:
y_train.sample(5, random_state=random_state)

Como el conjunto de datos de prueba:

In [ ]:
X_test.sample(5, random_state=random_state)

In [ ]:
y_test.sample(5, random_state=random_state)

# 3. Análisis exploratorio de datos (práctica 1)

## 3.1. Descripción del conjunto de datos

Primero vamos a volver a unir los conjuntos de `X_train` e `y_train` en `data_train` para poder realizar las gráficas correctamente (`X_test` e `y_test` no los uniremos ya que no los vamos a necesitar). Además, comprobamos que la unión se realiza correctamente:

In [ ]:
data_train = utils_ex.join_dataset(X_train, y_train)

data_train.sample(5, random_state=random_state)

Una vez unidos, vamos a comprobar cuáles son las variables de la base de datos.

In [ ]:
X_train.info(memory_usage=False)

In [ ]:
y_train.value_counts()

Podemos ver que contamos con 398 instancias, las cuales tienen las 30 variables predictoras numéricas que hemos mencionado en el apartado 2 y la variable clase `diganosis`, que cuenta con dos posibles estados (`B` y `M`). También podemos observar que las variables predictoras no tienen ningún valor perdido, y que la variable clase está desbalanceada, ya que tiene casi el doble de instancias con clase `B` (benigno) que con `M` (maligno).

---

Por último, vamos a ver númericamente cómo se distribuyen los datos de las variables predictoras (máximo valor, mínimo, media, desviación estándar...):

In [ ]:
X_train.describe(include="number").T

Podemos observar que los datos están en rangos muy distintos para cada variable, teniendo por ejemplo `area_worst` una media de 900.581910 mientras que `concave points_mean` tienen una media de 0.049212.

# 4. Preprocesamiento de datos (práctica 1)

Aquí vamos a definir el preprocesamiento que vamos a realizar a nuestros datos teniendo en cuenta las conclusiones a las que llegamos en el análisis exploratorio de la práctica 1. En la selección de modelos podremos comparar la diferencia entre usar y no usar este preprocesamiento o algunas partes del mismo, definiendo los hiperparámetros de los *pipelines*.

En el análisis exploratorio, vimos que había una gran cantidad de **variables muy correlacionadas entre sí**, por lo que puede ser beneficioso eliminar algunas de ellas. Esto hará que algunos algoritmos especialmente sensibles a los datos redundantes puedan mejorar sus clasificaciones.

Por otro lado, también hemos visto que **no había valores perdidos en las variables**, por lo que no necesitaremos imputarlos.

Lo que sí que encontramos fueron ***outliers***, por lo que podremos eliminarlos para intentar mejorar el rendimiento de algunos algoritmos que se vean especialmente perjudicados por los mismos.

Y por último, vimos que en algunas variables había divisiones bastante claras en las mismas que conseguían dejar en cada una casi todas las instancias de la misma clase, por lo que puede ser buena idea realizar una **discretización**.

---

### 4.1. Eliminar variables

Para eliminar las variables, en este caso (y al contrario de la práctica 1, en la que directamente eliminábamos las variables que tenían mayor correlación en el análisis exploratorio) vamos a utilizar `SelectKBest`, una función que nos seleccionará las `k` (por defecto 10) mejores variables ordenadas por la función que se le indique (por defecto `f-classif`, el *ANOVA F-value* entre la variable y las etiquetas de la clase).

In [ ]:
skb = SelectKBest()

### 4.2. Eliminar outliers

Para eliminar los *outliers* vamos a usar una función (`outlier_rejection`) que cree un `IsolationForest` para detectarlos, y después pasarla por un `FunctionSampler` para poder incluirla en el pipeline. Además, le hemos añadido el parámetro `eliminar` respecto a la práctica 1, para así poder seleccionar la eliminación o no de los outliers estableciendo dicho hiperparámetro a True o False respectivamente.

In [ ]:
def outlier_rejection(X, y, eliminar):
    if eliminar:
        model = IsolationForest(max_samples=100,
                                contamination=0.4,
                                random_state=random_state)
        model.fit(X)
        y_pred = model.predict(X)
        return X[y_pred == 1], y[y_pred == 1]
    else:
        return X, y

elimOutliers = FunctionSampler(func=outlier_rejection)

Sin embargo, en las pruebas realizadas parece que no hay ningún cambio entre usar dicha función o no usarla, por lo que la hemos eliminado de los *pipelines* y del *GridSearch* para así obtener además un tiempo de ejecución mucho menor.

### 4.3. Discretización

___

En la práctica 1 utilizamos un discretizador por `n_bins=2`, ya que vimos en el análisis de las variables que muchas de ellas como `area_mean`, `area_se`, `concavity_mean` o `concavity_worst` se pueden dividir casi perfectamente en dos partes, dejando a cada lado la clase mayoritaria. Además, usamos `strategy="kmeans"` (las otras opciones son `uniform` y `quantile`) porque como los datos están desbalanceados, si partiésemos por ejemplo por la media seguramente esa partición sería peor. 

Sin embargo, en esta práctica los valores de `n_bins` y `strategy` también pueden ser incluidos en la selección de parámetros, por lo que vamos a dejar su valor por defecto.

In [ ]:
discretizer = KBinsDiscretizer()

# 5. Modelos de clasificación supervisada

Ahora vamos a definir qué modelos de clasificación supervisada vamos a usar, y los pipelines que utilizaremos. En los mismos incluiremos tanto a los modelos como al preprocesamiento.

---

## 5.1. Vecinos más cercanos

El algoritmo de los vecinos más cercanos es un método basado en instancias que no construye ningún modelo durante el aprendizaje, pues directamente almacena las instancias del conjunto de datos de entrenamiento. Además se considera perozoso dado que computa los parámetros necesarios para la clasificación durante inferencia. En particular, la clasificación consiste en asignar a la instancia de entrada la clase mayoritaria de los vecinos más cercanos.

Este clasificador se implementa en la clase `neighbors.KNeighborsClassifier`. Es muy fácil de configurar, pues son dos los hiperparámetros más importantes a fijar:

* `n_neighbors`: Número de vecinos más cercanos (por defecto, `n_neighbors=5`). El número de vecinos más cercanos provoca que el clasificador sea más robusto al ruido, pero al mismo tiempo se obtienen fronteras de decisión menos distinguidas.

* `weights`: Función de pesado de los vecinos más cercanos (por defecto, `weights="uniform"`). Si bien lo más sencillo es considerar que todos los vecinos más cercanos tienen la misma importancia (`weights="uniform"`), es útil pesarlos de acuerdo con la inversa de la distancia a la instancia de entrada (`weights="distance"`).

Como se ha comentado, el aumento del número de vecinos hace que el clasificador sea más robusto al ruido. Esto servirá para intentar evitar en la medida de lo posible el sobreajuste a los datos de entrenamiento, aunque esta sea una tarea compleja con este algoritmo.

---

Vamos a inicializar el clasificador de los vecinos más cercanos con todos sus hiperparámetros por defecto, y dos *pipelines*: 
* `k_neighbors_model`, que incluye la eliminación de variables, el discretizador y el modelo de vecinos más cercanos.
* `k_neighbors_modelSD`,que incluye la eliminación de variables y el modelo de vecinos más cercanos.

In [ ]:
k_neighbors = KNeighborsClassifier()

k_neighbors_model = make_pipeline(skb,
                                  discretizer,
                                  k_neighbors)

k_neighbors_modelSD = make_pipeline(skb,
                                    k_neighbors)

## 5.2. Árboles de decisión

Los algoritmos basados en inducción de árboles de decisión representan los modelos mediante un conjunto de reglas. Estos presentan una serie de ventajas tal y como puede ser la interpretabilidad de los modelos obtenidos, bajo coste del proceso de predicción (logarítmico con respecto al número de muestras del conjunto de datos de prueba), manejo implícito de variables numéricas y categóricas, etc. Por contra, una de las mayores desventajas es que tienden a crear modelos demasiado complejos que no suelen generalizar ante conjuntos de datos no visualizados por el algoritmo de aprendizaje. Entre otros problemas destacan la inestabilidad de los modelos obtenidos ante variaciones del conjunto de datos de entrenamiento, dificultad para modelar determinados tipos de problemas (p.e., XOR), etc.

Los árboles de decisión se encuentran implementados en la clase `tree.DecisionTreeClassifier`. Los hiperparámetros más importantes para ajustarlos son:

* `criterion`: Criterio utilizado para medir la calidad de una partición (por defecto, `criterion="gini"`).
* `max_depth`: Altura máxima del árbol de decisión (por defecto, `max_depth=None`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).
* `ccp_alpha`: Parámetro de complejidad usado para el algoritmo de post-poda *Minimal Cost-Complexity Pruning* (por defecto, `ccp_alpha=0.0`).

Los valores por defecto de los hiperparámetros que controlan el tamaño del árbol de decisión (`max_depth`, `min_samples_split`, etc.) producen árboles de decisión profundos (*fully-developed decision trees*) que suelen estar sobreajustados al conjunto de datos de entrenamiento. Por ello, es recomendable configurar estos hiperparámetros de acuerdo con el problema a ser resuelto, evitando así que el árbol se expanda demsiado (con `max_depth` o `min_samples_split`, por ejemplo) o podándolo una vez que se ha expandido (con `ccp_alpha`).

Sin embargo, este tipo de árboles profundos sí que serán útiles posteriormente ya que se pueden combinar muchos de ellos con un *ensemble* para reducir su varianza y obtener un modelo generalmente mejor. Esto es lo que haremos posteriormente en el algoritmo de *Bagging* y *Random Forest*.

Aunque también podemos utilizar la técnica contraria, realizando un *ensemble* utilizando árboles de decisión poco profundos y reduciendo su sesgo mediante la realización de múltiples iteraciones en las que las instancias no tienen los mismos pesos para así centrarse en los valores más difíciles de generalizar. Esto es lo que haremos con *AdaBoost*, *Gradient Boosting* e *Histogram Gradient Boosting*.

---

Vamos a inicializar el árbol de decisión con todos sus hiperparámetros por defecto (menos la semilla a usar), y dos *pipelines*: 
* `decision_tree_model`, que incluye la eliminación de variables, el discretizador y el modelo del árbol de decisión.
* `decision_tree_modelSD`,que incluye la eliminación de variables y el modelo del árbol de decisión.

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=random_state)

decision_tree_model = make_pipeline(skb,
                                    discretizer,
                                    decision_tree)

decision_tree_modelSD = make_pipeline(skb,
                                      decision_tree)

## 5.3. *Adaptative Boosting* (*AdaBoost*)

Aunque la estrategia común detrás de un ensemble sea la de generar un consenso entre varios modelos, la forma en la que se generan está dirigida por motivaciones muy diferentes. Cuando utilizamos una técnica de ensemble es básico conocer el fundamento interno para seleccionar adecuadamente los hiperparámetros del algoritmo de aprendizaje.

En el caso de los algoritmos de *Boosting*, la estrategia es, dado un problema complejo, reducir el sesgo de los modelos. Por ello, lo que se hace es guiar la función de aprendizaje para que incida en aquellos ejemplos que sean más dificiles de generalizar. La técnica es pesar las instancias para sobreajustar el clasificador en aquellos casos conflictivos. De manera más formal se trata de establecer un problema de optimización de funciones de coste.

Por ello, los clasificadores ideales que se pueden utilizar en técnicas de *Boosting* son *weak learners*, esto es, clasificadores con un poder de generalización y parámetros no muy complejos ni sobreajustados. Esto otorga al algoritmo espacio para optimizar dichos parámetros y teóricamente transformar el clasificador base en un *strong learner*. Por ejemplo, árboles de decisión poco profundos (*shallow decision trees*).

El algoritmo *AdaBoost* está disponible en la clase `ensemble.AdaBoostClassifier` cuyos hiperparámetros más relevantes son:

* `base_estimator`: Estimador base que usa el ensemble (por defecto, `base_estimator=None`). Si no se especifica, se utiliza un árbol de decisión de profundidad uno.
* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=50`).
* `learning_rate`: Parámetro de regularización para controlar la contribución de cada estimador (por defecto, `learning_rate=1.0`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

---

Vamos a configurar un modelo *AdaBoost* sencillo con todos sus hiperparámetros por defecto (excepto la semilla), y dos *pipelines*: 
* `adaboost_model`, que incluye la eliminación de variables, el discretizador y el modelo *AdaBoost*.
* `adaboost_modelSD`,que incluye la eliminación de variables y el modelo *AdaBoost*.

In [ ]:
adaboost = AdaBoostClassifier(random_state=random_state)

adaboost_model = make_pipeline(skb,
                               discretizer,
                               adaboost)

adaboost_modelSD = make_pipeline(skb,
                                 adaboost)

## 5.4. *Bootstrap Aggregating* (*Bagging*)

En el ensemble *Bootstrap Aggregating* (*Bagging*), la estrategia es utilizar una función de aprendizaje y obtener modelos bien diversos entre sí para reducir el error obtenido mediante varianza. Formalmente, hablamos de clasificadores que individualmente tengan poder de generalización y un gran poder predictivo, pero que al mismo tiempo estén lo menos correlados entre sí. Por ello, se utilizan diversas técnicas de muestreo y aleatorización de los modelos.

De esta manera, el mejor tipo de clasificadores que se pueden utilizar con *Bagging* son *strong learners*, esto es, clasificadores con un gran poder predictivo en sus parámetros y modelos que presenten mucha varianza para reducirla mediante agregación. Por ejemplo, árboles de decisión profundos.

*Bagging* se encuentra implementado en la clase `ensemble.BaggingClassifier` y sus hiperparámetros más importantes son:

* `base_estimator`: Estimador base que usa el ensemble (por defecto, `base_estimator=None`). Si no se especifica, se utiliza un árbol de decisión sin podar.
* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=10`).
* `max_samples`: Número o proporción de muestras a utilizar en el muestreo (por defecto, `max_samples=1.0`).
* `max_features`: Número o proporción de características a utilizar en el muestreo (por defecto, `max_features=1.0`).
* `bootstrap`: Cuándo el muestreo de instancias es con reemplazo (por defecto, `bootstrap=True`).
* `bootstrap_features`: Cuándo el muestreo de características es con reemplazo (por defecto, `bootstrap_features=False`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

---

Vamos a configurar un ensemble tipo *Bagging* básico con todos sus hiperparámetros por defecto (excepto la semilla), y dos *pipelines*: 
* `bagging_model`, que incluye la eliminación de variables, el discretizador y el modelo *Bagging*.
* `bagging_modelSD`, que incluye la eliminación de variables y el modelo *Bagging*.

In [ ]:
bagging = BaggingClassifier(random_state=random_state)

bagging_model = make_pipeline(skb,
                              discretizer,
                              bagging)

bagging_modelSD = make_pipeline(skb,
                                bagging)

## 5.5. *Random Forests*

Una alternativa muy popular a *Bagging* es *Random Forests*. En este caso, también se busca reducir el error obtenido mediante varianza, pero adicionalmente integra otras técnicas de aleatorización en el aprendizaje de los árboles de decisión (no requiere un clasificador base para su definición) para ampliar la generalización del ensemble. Concretamente, utiliza una muestra aleatoria de los atributos a la hora de seleccionar cada punto óptimo de corte. Por esa misma razón, los hiperparámetros de este algoritmo incluyen tanto los correspondientes al ensemble como al árbol de decisión.

El ensemble *Random Forests* se implementa en la clase `ensemble.RandomForestClassifier` y recopilando los hiperparámetros más importantes tenemos:

* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=100`).
* `criterion`: Criterio utilizado para medir la calidad de una partición (por defecto, `criterion="gini"`).
* `max_features`: Número de características a considerar en cada nodo del árbol (por defecto, `max_features="auto"`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

---

Vamos a configurar un estimador tipo *Random Forests* básico con todos sus hiperparámetros por defecto (menos la semilla), y dos *pipelines*: 
* `random_forest_model`, que incluye la eliminación de variables, el discretizador y el modelo *Random Forests*.
* `random_forest_modelSD`, que incluye la eliminación de variables y el modelo *Random Forests*.

In [ ]:
random_forest = RandomForestClassifier(random_state=random_state)

random_forest_model = make_pipeline(skb,
                                    discretizer,
                                    random_forest)

random_forest_modelSD = make_pipeline(skb,
                                      random_forest)

## 5.6. *Gradient Tree Boosting* (*Gradient Boosting*)

*Gradient Tree Boosting* (*Gradient Boosting*) es una generalización de los algoritmos de *Boosting* con la capacidad de optimizar cualquier tipo de función pérdida. Esto permite extender estos algoritmos más allá de problemas de clasificación binaria tal y como pueden ser problemas de regresión, etc.

Este algoritmo está implementado en la clase `ensemble.GradientBoostingClassifier` y los hiperparámetros más importantes son:

* `loss`: Función de pérdida a optimizar (por defecto, `loss="deviance"`).
* `learning_rate`: Parámetro de regularización para controlar la contribución de cada estimador (por defecto, `learning_rate=0.1`).
* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=100`).
* `criterion`: Criterio utilizado para medir la calidad de una partición (por defecto, `criterion="friedman_mse"`). 
* `max_depth`: Altura máxima de los árboles de decisión (por defecto, `max_depth=3`).
* `ccp_alpha`: Parámetro de complejidad usado para el algoritmo de post-poda *Minimal Cost-Complexity Pruning* (por defecto, `ccp_alpha=0.0`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

---

Vamos a configurar un estimador tipo *Random Forests* básico con todos sus hiperparámetros por defecto (menos la semilla), y dos *pipelines*: 
* `gradient_boosting_model`, que incluye la eliminación de variables, el discretizador y el modelo *Gradient Boosting*.
* `gradient_boosting_modelSD`, que incluye la eliminación de variables y el modelo *Gradient Boosting*.

In [ ]:
gradient_boosting = GradientBoostingClassifier(random_state=random_state)

gradient_boosting_model = make_pipeline(skb,
                                        discretizer,
                                        gradient_boosting)

gradient_boosting_modelSD = make_pipeline(skb,
                                          gradient_boosting)

## 5.7. *Histogram-Based Gradient Boosting* (*Histogram Gradient Boosting*)

El algoritmo *Histogram-Based Gradient Boosting* (*Histogram Gradient Boosting*) es una optimización de *Gradient Boosting* que discretiza el conjunto de datos de entrada para reducir el número de puntos de corte a considerar en la construcción de los árboles de decisión (aprovechando las estructuras de datos basadas en histogramas). De esta manera, no tiene que considerar cada valor distinto de las variables predictoras continuas como punto de corte al construir los árboles de decisión, lo que le permite reducir en varios órdenes de magnitud el tiempo de entrenamiento e inferencia. Otra de las ventajas que presenta es que es capaz de tratar valores perdidos de manera implícita sin necesidad de realizar una imputación previa.

El algoritmo *Histogram Gradient Boosting* se implementa en la clase `ensemble.HistGradientBoostingClassifier` y cuyos hiperparámetros más básicos son:

* `loss`: Función de pérdida a optimizar (por defecto, `loss="auto"`).
* `learning_rate`: Parámetro de regularización para controlar la contribución de cada estimador (por defecto, `learning_rate=0.1`).
* `max_iter`: Número máximo de iteraciones del algoritmo (por defecto, `max_iter=100`).
* `max_leaf_nodes`: Número máximo de nodos hoja (por defecto, `max_leaf_nodes=31`).
* `max_depth`: Altura máxima de los árboles de decisión (por defecto, `max_depth=None`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

**Nota**: Este estimador es todavía experimental y se tiene que activar explícitamente con `experimental.enable_hist_gradient_boosting`.

---

Vamos a configurar el algoritmo *Histogram-Based Gradient Boosting* básico con todos sus hiperparámetros por defecto (menos la semilla), y esta vez un único *pipeline* (`hist_gradient_boosting_modelSD`) ya que en este caso el propio algoritmo es el que realiza la discretización. Por tanto, este *pipeline* contará con la eliminación de variables, el discretizador y el modelo *Histogram Gradient Boosting*.

In [ ]:
hist_gradient_boosting = HistGradientBoostingClassifier(random_state=random_state)

hist_gradient_boosting_modelSD = make_pipeline(skb,
                                               hist_gradient_boosting)

# 6. Evaluación de modelos

Ahora, tenemos que diseñar el proceso con el que vamos a elegir el mejor clasificador para el problema y a configurar los hiperparámetros de cada uno de los *pipelines* que hemos creado en el apartado anterior hiperparámetros. 

Para lograrlo, vamos a utilizar la validación cruzada. En ésta, se separa el conjunto de datos en $ k $ particiones y se repite $ k $ veces el proceso de aprendizaje y validación, pero utilizando cada vez una combinación única de $ k - 1 $ muestras para entrenar y la muestra restante para validar. El resultado es la media de las métricas obtenidas en cada una de las particiones. Al repetir y agregar los resultados del aprendizaje sobre varios conjuntos de datos diferentes, nos aseguramos una buena estimación del sesgo y la varianza. Además, de este modo utilizamos todas las instancias como test, por lo que el resultado se verá menos influido por la "suerte" de escoger como test instancias más o menos favorables.

Además, la validación cruzada que vamos a utilizar será estratificada, lo cual es muy necesario sobre todo cuando tenemos bases de datos cuya clase está desbalanceada. Así, el porcentaje de cada estado de la clase en cada una de las particiciones será lo más cercano posible al porcentaje de cada clase en los datos completos.

Para utilizar la validación cruzada usamos la función `model_selection.cross_validate` cuyos parámetros son:

* `estimator`: Estimador usado para ajustar los datos.

* `X`: Conjunto de variables predictoras.

* `y`: Variable clase.

* `scoring`: Métricas de rendimiento (por defecto, `scoring=None`). En problemas de clasificación se usa por defecto la tasa de acierto.

* `cv`: Método de validación cruzada a utilizar (por defecto, `cv=None`). Si no se especifica se usa una validación cruzada (estratificada) de 5 particiones. También puede tomar un número entero indicando el número de particiones que queremos utilizar para la validación cruzada (estratificada). Además se puede pasar el `splitter` que se quiere utilizar (`KFold`, `StratifiedKFold`, `RepeatedKFold`, `RepeatedStratifiedKFold`, etc.).

---

Su uso facilita mucho el realizar todo el proceso explicado, ya que solamente es necesario configurar el clasificador y el tipo de validación cruzada a utilizar. En nuestra selección de modelos posterior vamos a utilizar una $2 \times 10-cv$ estratificada:

**Nota:** Realizamos una $2 \times 10-cv$ estratificada en lugar de una $5 \times 10-cv$ estratificada ya que de esta forma podemos tener unos tiempos de ejecución más contenidos (2.5 veces menores), lo cual era relevante sobre todo teniendo en cuenta las dificultades añadidas ocasionadas por el límite de 9 horas seguidas de sesión en Kaggle ante una eventual unión de las dos libretas (que al final no hemos realizado).

In [ ]:
n_splits = 2
n_repeats = 10

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

# 7. Selección de modelos

---

Con la evaluación cruzada estratificada que hemos definido anteriormente, ya podemos evaluar correctamente los clasificadores. Sin embargo, aún tenemos que ver cómo podemos encontrar una configuración óptima de los hiperparámetros de cada uno de los modelos. Para ello, combinaremos un algoritmo de búsqueda con nuestra evaluación cruzada para así explorar el espacio de las distintas configuraciones de hiperparámetros de cada algoritmos de aprendizaje y del preprocesamiento.

Vamos a utilizar *Grid Search*, el algoritmo más básico de selección de modelos por fuerza bruta y que está implementado como un meta-estimador en `model_selection.GridSearchCV`. Este recibe un *grid* de hiperparámetros (diccionario con el nombre del hiperparámetro como clave y la lista de configuraciones como valores) y realiza una búsqueda exhaustiva evaluando mediante la validación cruzada que hemos definido anteriormente todas las posibles combinaciones existentes. Los hiperparámetros más relevantes son:

* `estimator`: Estimador del que se quieren optimizar sus hiperparámetros.
* `param_grid`: *Grid* de hiperparámetros.
* `scoring`: Métrica a optimizar en la búsqueda (por defecto, `scoring=None`).
* `cv`: Tipo de validación cruzada (por defecto, `cv=None`).

Las métricas que evaluaremos serán el `accuracy`, `AUC` (área bajo la *curva ROC*), `recall` y `precision`, ordenando posteriormente el *GridSearch* por la media de dichas métricas. Esto lo haremos por un lado porque nuestras bases de datos tienen como función predecir enfermedades (diabetes en un caso y cáncer en otro), por lo que es muy importante que no se nos escapen los casos positivos; y por otro lado como los datos están desbalanceados, una alta tasa de acierto puede significar simplemente que se están prediciendo todos las variables clases como la clase mayoritaria (lo que es lo mismo que un clasificador `Zero-R`, por lo que es necesario añadir otras métricas para evitar que ésto suceda.

Para seleccionar los hiperparámetros del preprocesamiento vamos a realizar dos ejecuciones de *Grid Search*: una discretizando, y otra sin discretizar. Cuando no discretizamos, no será necesario evaluar los hiperparámetros `n_bins` y `strategy`. Con los mejores hiperparámetros obtenidos de entre estas dos ejecuciones, pasaremos a realizar la construcción y validación del modelo final.

Los hiperparámetros del preprocesamiento que vamos a seleccionar van a ser:
* `kbinsdiscretizer__n_bins`: Número de intervalos en los que discretizar. Ya que hemos visto en el análisis exploratorio que había variables con las que se obtenían buenas particiones (por ejemplo, directamente por la media), vamos a usar un rango de *bins* bajo (2, 3, 4 y 5).
* `kbinsdiscretizer__strategy`: Estrategia para discretizar. Vamos a evaluar 2 de las 3 posibles: `uniform` y `kmeans`. `quantile` no la evaluamos ya que en test previos comprobamos que no era seleccionada por ningún algoritmo, y ni siquiera entraba en los mejores valores.
* `selectkbest__k`: Número de variables a seleccionar en *SelectKBeast*. Escogeremos los valores 5, 10, 15, 20, 25 y `all`, siendo `all` el total de las variables (30, en este caso no se realizaría la selección).

Además, estas ejecuciones estarán marcadas por las limitaciones que implica utilizar un *GridSearch* cuando hay que ajustar varios hiperparámetros, ya que por cada uno de ellos que añadamos habrá que multiplicar el número de ejecuciones (y con ello el tiempo) por el número de distintos valores que tome dicho hiperparámetro. Esto hará que no comprobemos algunos hiperparámetros que a priori también podrían ser útiles, ya que intentamos que los que sí podamos probar por lo menos tengan suficientes valores para obtener un buen resultado.

---

## 7.1. Vecinos más cercanos

Vamos a empezar con el algoritmo de los vecinos más cercanos, realizando primero la selección de hiperparámetros tanto del preprocesamiento como del modelo usando discretización, y después sin usarla. Los hiperparámetros que podemos seleccionar para este algoritmo son:
* El número de vecinos más cercanos (`kneighborsclassifier__n_neighbors)`: Valores impares, de 3 (1 no porque sobreajustaría demasiado) a 19 (ya que 20 es la raíz cuadrada del número de instancias de test).
* Función de pesado para la predicción (`kneighborsclassifier__weights`): `uniform` o `distance`.

Por tanto, vamos a empezar realizando la selección usando discretización:

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = k_neighbors_model

# Hiperparámetros a seleccionar
n_bins = [2, 4]
strategy = ['uniform', 'kmeans']
k = [10, 15, 25, 'all']

weights = ["uniform", "distance"]
n_neighbors = [3,5,7,9,11,13,15,17,19]

k_neighbors_clf = utils.optimize_params(estimator,
                                        X_train, y_train, cv,
                                        kbinsdiscretizer__n_bins=n_bins,
                                        kbinsdiscretizer__strategy=strategy,
                                        selectkbest__k=k,
                                        kneighborsclassifier__weights=weights,
                                        kneighborsclassifier__n_neighbors=n_neighbors)

Y sin usarla:

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = k_neighbors_modelSD

# Hiperparámetros ya seleccionados
k = [10, 15, 25, 'all']

# Hiperparámetros a seleccionar
weights = ["uniform", "distance"]
n_neighbors = [3,5,7,9,11,13,15,17,19]

k_neighbors_clfSD = utils.optimize_params(estimator,
                                        X_train, y_train, cv,
                                        selectkbest__k=k,
                                        kneighborsclassifier__weights=weights,
                                        kneighborsclassifier__n_neighbors=n_neighbors)

Una vez obtenida las salidas, nos quedamos con el conjunto de hiperparámetros con mejor media, que en este caso será discretizando. Dichos hiperparámetros son:

* `kbinsdiscretizer__n_bins = 4`: La discretización la haremos en 4 *bins*.
* `kbinsdiscretizer__strategy = uniform`: Discretizaremos uniformemente.
* `selectkbest__k = 25`: Nos quedamos con las 25 variables más importantes.
* `kneighborsclassifier__n_neighbors = 7`: Utilizamos los 7 vecinos más cercanos.
* `kneighborsclassifier__weights = distance`: Pesamos de acuerdo a la distancia inversa.

Por tanto, la tasa de acierto en los datos de test que conseguimos es de $0.945226 \pm 0.011337$ (la más alta), mientras que su *AUC* es $0.979305 \pm 0.006636$, su *Recall* $0.882432 \pm 0.038246$ y su  *Precision* $0.968373 \pm 0.020461$. Estas tres últimas métricas se quedan algo lejos de las mejores en el ranking, pero su media es buena. Esto va a ser algo normal en todas las ejecuciones, ya que es raro que un conjunto de hiperparámetros sea mejor en todas las métricas, sino que normalmente al mejorar unas empeorará algo otras.

También podemos ver un tremendo sobreajuste a los datos de test, ya que este es un algoritmo muy propenso a ello aun no seleccionando un número de vecinos más cercanos bajo.

---

## 7.2. Árboles de decisión



Continuando con los árboles de decisión, vamos a realizar los mismos pasos que en el algoritmo de los vecinos más cercanos. Por tanto vamos a optimizar, aparte de los hiperparámetros del preprocesamiento:
* El criterio de partición (`decisiontreeclassifier__criterion`): `gini` o `entropy`.
* La altura máxima (`decisiontreeclassifier__max_depth`): 1, 2, 3, 4, 5, 10, 15 o 20 (siendo 20 la raíz del número de instancias de test).
* El parámetro de complejidad de la poda (`decisiontreeclassifier__ccp_alpha`): 0 (sin postpoda), 0.01 o 0.02. 

Además, podríamos haber optimizado algunos otros hiperparámetros para reducir en mayor medida el sobreajuste como:
* El número mínimo de muestras necesarias para dividir un nodo interno (`min_samples_split`).
* El número mínimo de muestras requerido para estar en un nodo de la hoja (`min_samples_leaf`). 

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = clone(decision_tree_model)

# Hiperparámetros a seleccionar
n_bins = [2, 4]
strategy = ['uniform', 'kmeans']
k = [10, 15, 25, 'all']

criterion = ["gini", "entropy"]
max_depth = [1,2,3,4,5,10,15,20]
ccp_alpha = [0.0, 0.01, 0.02]

decision_tree_clf = utils.optimize_params(estimator,
                                          X_train, y_train, cv,
                                          kbinsdiscretizer__n_bins=n_bins,
                                          kbinsdiscretizer__strategy=strategy,
                                          selectkbest__k=k,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha)

Y después sin discretizar:

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = clone(decision_tree_modelSD)

# Hiperparámetros a seleccionar
k = [10, 15, 25, 'all']

criterion = ["gini", "entropy"]
max_depth = [1,2,3,4,5,10,15,20]
ccp_alpha = [0.0, 0.01, 0.02]

decision_tree_clfSD = utils.optimize_params(estimator,
                                          X_train, y_train, cv,
                                          selectkbest__k=k,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha)

Una vez realizadas las ejecuciones, nos quedamos con el conjunto de hiperparámetros con mejor media, que al igual que en el caso anterior la obtenemos discretizando. Los hiperparámetros que seleccionamos son:

Por tanto, el modelo de árbol de decision que hemos seleccionado, junto con su preprocesamiento, cuentan con los siguientes hiperparámetros (discretizando):
* `kbinsdiscretizer__n_bins = 2`: La discretización la haremos en 2 *bins*.
* `kbinsdiscretizer__strategy = kmeans`: Discretizaremos mediante `kmeans`.
* `selectkbest__k = 25`: Nos quedamos con las 25 variables más importantes.
* `functionsampler__kw_args = {'eliminar': 'true'}`: Eliminamos los *outliers*.
* `decisiontreeclassifier__ccp_alpha = 0.0`: Estableceremos el parámetro de complejidad de la poda a 0.
* `decisiontreeclassifier__criterion = 'gini'`: El criterio será `gini`.
* `decisiontreeclassifier__max_depth = '4'`: La profundidad máxima del árbol será de 4 niveles.

Con ellos, la tasa de acierto en los datos de test que conseguimos es de $0.938191 \pm 0.014043$, mientras que su *AUC* es de $0.954811 \pm 0.015038$, su *Recall* $0.897297 \pm 0.031519$ y su *Precision* $0.935261 \pm 0.031607$. Al igual que pasaba con el algoritmo de los vecinos más cercanos, la tasa de acierto obtenida es la mejor, mientras que las otras tres métricas no se quedan demasiado lejos de las mejores.

Además, podemos ver cómo pese al discretizamiento y a contar únicamente 4 con niveles en los árboles, se produce sobreajuste en los datos de test. También es relevante que no se realize la postpoda.

---

## 7.3. *AdaBoost*


En *AdaBoost* vamos a seguir el mismo procedimiento. En este algoritmo vamos a optimizar, aparte de los hiperparámetros del preprocesamiento:
* La tasa de aprendizaje (`adaboostclassifier__learning_rate`, la estableceremos a tres posibles valores relativamente altos: 0.95, 0.975 y 1.0).
* Los hiperparámetros de los árboles de clasificación que genera: 
  * El criterio de partición (`adaboostclassifier__base_estimator__criterion`: `gini` o `entropy`).
  * La altura máxima (`adaboostclassifier__base_estimator__max_depth`: valores pequeños, del 1 al 4, ya que como comentamos nos interesan árboles pequeños).
  * El parámetro de complejidad de la poda (`adaboostclassifier__base_estimator__ccp_alpha`: 0 (sin postpoda) o 0.05). 
  
También podríamos haber optimizado `n_estimators`, el número de árboles generados por el *ensemble*.

Primero vamos a optimizar los hiperparámetros de la selección de variables discretizando:

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = adaboost_model

# Should not modify the base original model
base_estimator = clone(decision_tree)

# Hiperparámetros a seleccionar
n_bins = [2, 4]
strategy = ['uniform', 'kmeans']
k = [10, 15, 25, 'all']

learning_rate = [0.95, 0.975, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3, 4]
ccp_alpha = [0.0, 0.05]
base_estimator = [base_estimator]

adaboost_clf = utils.optimize_params(estimator,
                                     X_train, y_train, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     kbinsdiscretizer__n_bins=n_bins,
                                     kbinsdiscretizer__strategy=strategy,
                                     selectkbest__k=k,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha)

Y después sin discretizar:

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = adaboost_modelSD

# Should not modify the base original model
base_estimator = clone(decision_tree)

# Hiperparámetros a seleccionar
k = [10, 15, 25, 'all']

learning_rate = [0.95, 0.975, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3, 4]
ccp_alpha = [0.0, 0.05]
base_estimator = [base_estimator]

adaboost_clfSD = utils.optimize_params(estimator,
                                     X_train, y_train, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     selectkbest__k=k,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha)

En *AdaBoost*, al igual que en los dos modelos anteriores, también obtenemos una mejor media en las métricas discretizando. Así, la tasa de acierto que obtenemos es de $0.954271 \pm 0.012196$ (la mejor de todas), su *AUC* en los datos de test es $0.988341 \pm 0.004432$ (la séptima mejor), su *Recall* es $0.925676 \pm 0.028507$ y su *Precision* (la mejor también) es $0.951105 \pm 0.027397$.

Los hiperparámetros seleccionados tanto para el preprocesamiento como para el algoritmo de *AdaBoost* y su clasificador base son:
* `kbinsdiscretizer__n_bins = 4`: La discretización la haremos en 2 *bins*.
* `kbinsdiscretizer__strategy = kmeans`: Discretizaremos utilizando `kmeans`.
* `'selectkbest__k'= 25`: Nos quedamos con las 25 variables más importantes.
* `adaboostclassifier__learning_rate = 0.975`: Estableceremos la tasa de aprendizaje a 0.975.
* `adaboostclassifier__base_estimator__ccp_alpha = 0.0`: Estableceremos el parámetro de complejidad de la poda a 0 (sin postpoda).
* `adaboostclassifier__base_estimator__criterion = 'entropy'`: El criterio será la entropía.
* `adaboostclassifier__base_estimator__max_depth = '3'`: La profundidad máxima de los árboles será de 3 niveles.

En este caso, igual que ocurría en el algoritmo de los vecinos más cercanos, también tenemos un sobreajuste total a los datos de entrenamiento, y tampoco se realiza poda.

---

## 7.4. *Bagging*

Ahora tenemos que realizar el mismo procedimiento para *Bagging*. En *Bagging* únicamente vamos a optimizar el criterio de partición (`baggingclassifier__base_estimator__criterion`: `gini` o `entropy`) de los árboles de decisión y no otros como la complejidad de la poda o la profundidad máxima, ya que en este modelo nos interesa que los árboles se ramifiquen mucho para sobreajustarlos, y posteriormente mediante el *ensemble* reducir su varianza. Además, otro hiperparámetro que podríamos haber optimizado es `n_estimators`, el número de árboles generados por el *ensemble*.

Primero seleccionamos los hipeparámetros discretizando:

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = bagging_model

# Should not modify the base original model
base_estimator = clone(decision_tree)

# Hiperparámetros a seleccionar
n_bins = [2, 4]
strategy = ['uniform', 'kmeans']
k = [10, 15, 25, 'all']

criterion = ["gini", "entropy"]
base_estimator = [base_estimator]

bagging_clf = utils.optimize_params(estimator,
                                    X_train, y_train, cv,
                                    baggingclassifier__base_estimator=base_estimator,
                                    selectkbest__k=k,
                                    kbinsdiscretizer__n_bins=n_bins,
                                    kbinsdiscretizer__strategy=strategy,
                                    baggingclassifier__base_estimator__criterion=criterion)

Y posteriormente sin discretizar:

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = bagging_modelSD

# Should not modify the base original model
base_estimator = clone(decision_tree)

# Hiperparámetros a seleccionar
k = [10, 15, 25, 'all']

criterion = ["gini", "entropy"]
base_estimator = [base_estimator]

bagging_clfSD = utils.optimize_params(estimator,
                                    X_train, y_train, cv,
                                    baggingclassifier__base_estimator=base_estimator,
                                    selectkbest__k=k,
                                    baggingclassifier__base_estimator__criterion=criterion)

Esta vez sí que obtenemos mejor media sin discretizar (lo cual tiene sentido tratándose de un algoritmo que intenta sobreajustar los árboles). Así, la tasa de acierto obtenida en los datos de test es $0.946734\pm0.012851$, el área bajo ROC es $0.984451\pm0.007925$, el *Recall* $0.918919\pm0.031692$ y la *Precision* $0.937973\pm0.029983$.

Por tanto, los hiperparámetros seleccionados, sin realizar discretización son:
* `selectkbest__k = 25`: Dejamos 25 variables.
* `baggingclassifier__base_estimator__criterion = 'entropy'`: El criterio será la entropía.

Al igual que ocurría en algoritmos anteriores, en este caso obtenemos unos datos casi perfectos en los datos de entrenamiento, que luego bajan un poco en los del test, por lo que también seguimos sobreajustando.

---

## 7.5. *Random Forests*

Vamos a seleccionar los hiperparámetros para *Random Forest* igual que en los casos anteriores. Este algoritmo, que al final es un *Bagging* cuyos árboles son *RandomTrees*, es suficiente con optimizar el criterio de partición (`randomforestclassifier__criterion`: `gini` o `entropy`) y el número máximo de características a considerar en cada nodo de los árboles de decisión  (`randomforestclassifier__max_features`: `sqrt` o `log2`). También podríamos haber seleccionado el número de ároles generados por el *ensemble* (`n_estimators`).

Primero vamos a seleccionar los hiperparámetros discretizando:

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = random_forest_model

# Hiperparámetros a seleccionar
n_bins = [2, 4]
strategy = ['uniform', 'kmeans']
k = [10, 15, 25, 'all']

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]

random_forest_clf = utils.optimize_params(estimator,
                                          X_train, y_train, cv,
                                          selectkbest__k=k,
                                          kbinsdiscretizer__n_bins=n_bins,
                                          kbinsdiscretizer__strategy=strategy,
                                          randomforestclassifier__criterion=criterion,
                                          randomforestclassifier__max_features=max_features)

Y ahora sin discretizar:

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = random_forest_modelSD

# Hiperparámetros a seleccionar
k = [10, 15, 25, 'all']

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]

random_forest_clfSD = utils.optimize_params(estimator,
                                          X_train, y_train, cv,
                                          selectkbest__k=k,
                                          randomforestclassifier__criterion=criterion,
                                          randomforestclassifier__max_features=max_features)

En este caso, al igual que con *Bagging*, también obtenemos mejores resultados sin discretizar. Así, conseguimos un *Accuracy* en los datos de test de $0.956281\pm0.011688$, una *AUC* de $0.990862\pm0.004753$, un *Recall* de $0.929730\pm0.024022$ y una *Precision* de $0.952446\pm0.026130$.

Por tanto, hiperparámetros seleccionados tanto para el preprocesamiento como para el algoritmo de *Random Forests*, discretizando, para conseguir  son:
* `selectkbest__k = 25`: Nos quedamos con las 25 mejores variables.
* `randomforestclassifier__max_features = log2`: El número máximo de características por árbol será el logaritmo en base 2 de las características.
* `randomforestclassifier__criterion = 'entropy'`: El criterio será la entropía.

---


## 7.6. *Gradient Boosting*

Continuamos con *Gradient Boosting*. Para este clasificador, vamos a optimizar los siguientes hiperparámetros (además de los del preprocesamiento, como en todos los algoritmos):
* Parámetro de regularización (`gradientboostingclassifier__learning_rate`): 0.01, 0.05, 0.1 y 0.2. 
* Altura máxima de los árboles (`gradientboostingclassifier__max_depth`): 1, 2 y 3, todos ellos valores bajos como es habitual en *Boosting*.
* Parámetro de complejidad de la poda de los árboles de decisión (`gradientboostingclassifier__ccp_alpha`): 0.0 y 0.005.

**Nota:** No realizamos la selección del hiperparámetro `criterion` ya que, al menos con nuestros datos, no afecta a la salida.

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = gradient_boosting_model

# Hiperparámetros a seleccionar
n_bins = [2, 4]
strategy = ['uniform', 'kmeans']
k = [10, 15, 25, 'all']

learning_rate = [0.01, 0.05, 0.1, 0.2]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.005]

gradient_boosting_clf = utils.optimize_params(estimator,
                                              X_train, y_train, cv,
                                              kbinsdiscretizer__n_bins=n_bins,
                                              kbinsdiscretizer__strategy=strategy,
                                              selectkbest__k=k,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              gradientboostingclassifier__ccp_alpha=ccp_alpha)

In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = gradient_boosting_modelSD

# Hiperparámetros a seleccionar
k = [10, 15, 25, 'all']

learning_rate = [0.01, 0.05, 0.1, 0.2]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.005]

gradient_boosting_clfSD = utils.optimize_params(estimator,
                                              X_train, y_train, cv,
                                              selectkbest__k=k,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              gradientboostingclassifier__ccp_alpha=ccp_alpha)

Para *Gradient Boosting* obtenemos una media en las métricas mejor sin discretizar. Así, la tasa de acierto sería $0.961055\pm0.014291$, mientras que el área bajo la curva ROC sería $0.993557\pm0.002817$, el *Recall* $0.933784\pm0.031373$ y la *Precision* $0.961151\pm0.025771$.

Para ello, los hiperparámetros seleccionados tanto para el preprocesamiento como para el algoritmo de *Gradient Boosting* son (aunque hay un empate entre varias combinaciones):
* `selectkbest__k = 'all'`: Nos quedamos con todas las variables.
* `gradientboostingclassifier__learning_rate = 0.2`: El parámetro de regularización será 0,2.
* `gradientboostingclassifier__max_depth = 1`: La profundidad máxima de los árboles será de 1 nivel.
* `gradientboostingclassifier__ccp_alpha = 0.0'`: El parámetro de complejidad de la poda será de 0,0.

---

## 7.7. *Histogram Gradient Boosting*

Por último, vamos a realizar el procedimiento para *Histogram Gradient Boosting*. En este caso, ya que este algoritmo solo trabaja con variables continuas, solo vamos a seleccionar los hiperparámetros del preprocesamiento sin discretizar. Los hiperparámetros del modelo que vamos a seleccionar son:
* El parámetro de regularización (`histgradientboostingclassifier__learning_rate`): 0.1, 0.25 o 0.4.
* El número máximo de nodos hoja de los árboles de decisión (`histgradientboostingclassifier__max_leaf_nodes`): 15, 31, 65 o 127.
* El número máximo de iteraciones del algoritmo (`histgradientboostingclassifier__max_iter`): 50, 75, 100 o 125.
* La altura máxima de los árboles de decisión (`histgradientboostingclassifier__max_depth`): 1, 3 o 5, pequeños al tratarse de *Boosting*.


In [ ]:
# Estimador a optimizar sus hiperparámetros
estimator = hist_gradient_boosting_modelSD

# Hiperparámetros a seleccionar
k = [10, 15, 25, 'all']

learning_rate = [0.1, 0.25, 0.4]
max_leaf_nodes = [15, 31, 65, 127]
max_iter = [50, 75, 100, 125]
max_depth = [1, 3, 5]

hist_gradient_boosting_clfSD = utils.optimize_params(estimator,
                                                   X_train, y_train, cv,
                                                   selectkbest__k=k,
                                                   histgradientboostingclassifier__learning_rate=learning_rate,
                                                   histgradientboostingclassifier__max_leaf_nodes=max_leaf_nodes,
                                                   histgradientboostingclassifier__max_iter=max_iter,
                                                   histgradientboostingclassifier__max_depth=max_depth)


La tasa de acierto que obtendríamos en los datos de test es $0.963819\pm0.009211$, mientras que su *AUC* es $0.994489\pm0.002020$, el *Recall* $0.939189\pm0.022813$ y la *Precision* $0.963108\pm0.018876$.

Para conseguir estos valores, la configuración de hiperparámetros escogida es (de entre el empate en las primeras posicioens):
* `selectkbest__k = 'all'`: Nos quedamos con todas las variables.
* `histgradientboostingclassifier__learning_rate = 0.4`: El parámetro de regularización será 0,4.
* `histgradientboostingclassifier__max_depth = 1`: Los árboles tendrán una profundidad máxima de 1.
* `histgradientboostingclassifier__max_iter = 100`: Se realizarán 100 iteraciones como máximo.
* `histgradientboostingclassifier__max_leaf_nodes' = 15`: El el número máximo de nodos hoja será 15.

Con este algoritmo también conseguimos un pleno de aciertos en los datos de test, pero en este caso para los datos de entrenamiento también obtenemos un mejor valor en todas las métricas que con algoritmos anteriores, por lo que existe sobreajuste pero no es tan grande como en otros modelos. También hay que comentar que el parámetro `max_leaf_nodes` parece no tener importancia, por lo menos con el conjunto de datos con el que estamos trabajando.

---

# 8. Construcción y validación del modelo final

En un entorno de producción se debe construir un clasificador que será el que se despliegue y use para clasificar nuevas instancias. Es importante saber que, después de un proceso de validación cruzada para la optimización de los hiperparámetros, se debe obtener un modelo final y una medida generalizada de su rendimiento.

Para ello, nos surgen dos dudas:

* La validación cruzada crea `k` modelos, ¿con cuál nos quedamos?
* La medida de rendimiento obtenida es ¿la media de la validación cruzada? o ¿para una partición concreta?

La solución pasa por utilizar el conjunto de datos de prueba que se ha quedado fuera de la validación cruzada. De este modo, realizaremos la validación cruzada a partir del conjunto de datos de entrenamiento para seleccionar nuestro modelo. Una vez se ha obtenido la mejor configuración de hiperparámetros (de acuerdo con la métrica de rendimiento utilizada durante este proceso), utilizaremos el conjunto de datos de entrenamiento para entrenar un nuevo clasificador final a partir de dicha configuración. Para validar este clasificador y obtener una métrica no sesgada de su rendimiento, utilizaremos el conjunto de datos de prueba que separamos al principio y que constituye una muestra de datos nunca visualizada por el  algoritmo de aprendizaje.

Este proceso se realiza automáticamente en el meta-estimador `model_selection.GridSearchCV`, por lo que podemos utilizarlo directamente para obtener una métrica de rendimiento final para cada uno de los clasificadores:

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clfSD,
    "Random Forests": random_forest_clfSD,
    "Gradient Boosting": gradient_boosting_clfSD,
    "Histogram Gradient Boosting": hist_gradient_boosting_clfSD,
}

In [ ]:
utils.evaluate_estimators(estimators, X_test, y_test)

A la vista de los resultados, podemos obtener las siguientes conclusiones:
* Claramente, los **árboles de decisión** y ***Bagging*** son los algoritmos que peores resultados nos dan en casi todas las métricas.
* El algoritmo de los **vecinos más cercanos** no obtiene malos valores de *accuracy* y *AUC*, pero su *recall* es demasiado malo en comparación con otros algoritmos, siendo muy relevante en una base de datos como la que estamos tratando, en la cual tenemos que predecir si un cancer es benigno o maligno por lo que nos interesa detectar todos los casos malignos posibles.
* De entre los algoritmos basados en *Bagging*, ***Random Forests*** funciona bastante mejor que el *Bagging* normal. Además, obtiene resultados de los mejores en comparación con el resto de algoritmos.
* De los algoritmos basados en *Boosting*, parece haber un orden de peores a mejores resultados entre ***AdaBoost***, ***Gradient Boosting*** e ***Histogram Gradient Boosting***. Estos algoritmos junto con *Random Forests* parecen los mejores para este problema.
* Entre *Random Forests* e *Histogram Gradient Boosting*, obtenemos resultados muy similares con la diferencia de que *Random Forests* obtiene una mejor *Precision* mientras que *Histogram Gradient Boosting* obtiene un mejor *Recall*, 

Viendo todo esto, nos vamos a quedar como ganador con ***Histogram Gradient Boosting***, ya que consigue un *Recall* bastante superior al del resto de algoritmos mientras que su *Accuracy* y *AUC* también son las mejores (aunque empatada con *Random Forests* en el primer caso), y aunque su *Precision* sí que es superada por bastante márgen, no es tan importante en este problema como puede ser *Recall* ya que, como hemos comentado, es preferible detectar más tumores malignos correctamente aunque sea a costa de que detectemos también algún tumor benigno como malingo.


